In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from scipy import stats
import chart_studio.plotly as py
import plotly.express as px
import cufflinks as cf
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

In [ ]:
name_df = pd.read_csv("https://datasets.imdbws.com/name.basics.tsv.gz", sep="\t")
name_df.head(1)

In [ ]:
principals_df = pd.read_csv("https://datasets.imdbws.com/title.principals.tsv.gz", sep="\t")
principals_df.head(1)

In [ ]:
basics_df = pd.read_csv("https://datasets.imdbws.com/title.basics.tsv.gz", sep="\t", low_memory=False)
basics_df.head(1)

In [ ]:
# Etape 1 : nettoyage de la Database
principals_df2 = principals_df[["tconst", "nconst", "category", "characters"]]
principals_df3 = principals_df2[principals_df2['category'].str.contains('actor|actress') & (principals_df2['characters'] != '\\N')]
principals_df4 = principals_df3[(principals_df3['characters'] != 'Narrator') & (principals_df3['characters'] != 'Various') & (principals_df3['characters'] != 'Additional Voices')]

In [ ]:
# Nettoyage de la Database
basics_df2 = basics_df[["tconst", "titleType", "startYear", "runtimeMinutes", "genres"]]
# Etape 2 : set index puis merge
# Set index on tconst for basics and principal table
basics_df3 = basics_df2.set_index('tconst')
principals_df5 = principals_df4.set_index('tconst')
# Merge de la table principals et basics afin de filtrer uniquement sur les movies et les shorts
actors_occurence_df = pd.merge(basics_df3, principals_df5, how='inner', left_index=True, right_index=True)

In [ ]:
actors_occurence_df["titleType"].value_counts()

In [ ]:
# Etape 3 : Nettoyage pour garder uniquement les titleType movie et short
#actors_occurence_df2 = actors_occurence_df[(actors_occurence_df['titleType'] == 'movie') | (actors_occurence_df['titleType'] == 'short')]
actors_occurence_df2 = actors_occurence_df[(actors_occurence_df['titleType'] == 'tvEpisode') | (actors_occurence_df['titleType'] == 'tvSeries') | (actors_occurence_df['titleType'] == 'tvMiniSeries')]

In [ ]:
# Nettoyage de la Database
name_df2 = name_df[["nconst", "primaryName"]]
# Etape 4 : set index puis merge
# Set index on nconst for Database name and Database actors_occurence_df2 to prepare a merge
name_df3 = name_df2.set_index('nconst')
actors_occurence_df3 = actors_occurence_df2.set_index('nconst')
# Merge to link first name with nconst
actors_occurence_df4 = pd.merge(name_df3, actors_occurence_df3, how='inner', left_index=True, right_index=True)

In [ ]:
# Etape 5 : Nettoyage pour retirer les oeuvres très courtes comme les cartoons de quelques minutes pour lesquelles les acteurs font seulement les voix
actors_occurence_df5 = actors_occurence_df4[actors_occurence_df4['runtimeMinutes'] != '\\N']
actors_occurence_df6 = actors_occurence_df5.astype({"runtimeMinutes": int})
actors_occurence_df7 = actors_occurence_df6[actors_occurence_df6['runtimeMinutes'] > 20]

In [ ]:
# Etape 6 : Value_counts pour trouver les acteurs les plus présents
actors_occurence_df8 = actors_occurence_df7['primaryName'].value_counts()
actors_occurence_df9 = actors_occurence_df8.head(20)

In [ ]:
# Etape 7 : transformation d'une série en dataframe puis reset de l'index
actors_occurence_df10 = pd.DataFrame(actors_occurence_df9)
actors_occurence_df10.reset_index(inplace=True)

In [ ]:
# # Etape 8 : Sauvegarde du dataframe
actors_occurence_df10.to_csv(r'C:\Users\Berenger\Desktop\projet abc\presence_acteurs.csv', index = False, header = True)

In [3]:
# # Etape 9 : Chargement du dataframe
presence_acteur = pd.read_csv(r"C:\Users\Berenger\Desktop\projet abc\presence_acteurs.csv")
presence_acteur.head()

,index,primaryName
0,Mohanlal,236
1,Eric Roberts,227
2,Mammootty,224
3,Cüneyt Arkin,219
4,Raymond Hatton,202


In [4]:
# étape 10 : visualisation
fig = px.bar(presence_acteur, x="primaryName", y ='index', color = 'index',
    title = 'Quels sont les acteurs les plus présents ?',
    labels = {'primaryName': 'Nombre de films', 'index': 'Acteurs'},
    width=800, height=600)

fig.update_layout(showlegend=False, title_x=0.5, yaxis={'visible': False})